In [ ]:
# system libraries
import os
import sys

# check system information
print('Python Information', sys.version)
print('This is your current directory', os.getcwd())

In [ ]:
# datetime libraries
import datetime

# assgin current date and time
currentDate = datetime.date.today()
currentTime = datetime.datetime.now()

# check datetime information
print('Today is {}'.format(currentDate))
print('Today is', datetime.datetime.strftime(currentDate, '%m/%d/%Y'))
print('The time is', datetime.datetime.strftime(currentTime, '%H:%M:%S'))

In [ ]:
!pip install pymongo

In [ ]:
import json
from pprint import pprint
import csv
import random

from bson.son import SON
from bson import json_util

In [ ]:
URL = 'https://data.ny.gov/api/views/3x8r-34rs/rows.csv?accessType=DOWNLOAD&sorting=true'

print(URL)

In [ ]:
filename = '/Users/ppfor/Downloads/Solar_Electric_Programs_Reported_by_NYSERDA__Beginning_2000.csv'

with open(filename, newline="", encoding='utf-8') as f:
    solar_data = list(csv.reader(f, delimiter = ","))
    solar_header = solar_data.pop(0)
    
print("Total rows: {:,}".format(len(solar_data)))

In [ ]:
for i, x in enumerate(random.choice(solar_data)):
    print(i, solar_header[i], ":", x)

In [ ]:
def nums(x):
    try:
        return float(x)
    except ValueError:
        return 0
    
test = ['', '55', 500, 'five', '5_000', '5e6']

list(map(nums, test))

In [ ]:
def dates(x):
    try:
        return datetime.datetime.strptime(x, '%m/%d/%Y')
    except:
        return None
    
test = ['7/31/2016', '', 'None', 'July 31 2016']

for item in list(map(dates, test)):
    print(item)

In [ ]:
def ymd(x):
    try:
        y = x.split("/")[-1]
        m = x.split("/")[0]
        d = x.split("/")[1]
        return SON(zip(('year', 'month', 'day'), (int(y), int(m), int(d))))
    except:
        return None
    
for item in test:
    print(ymd(item))

In [ ]:
solar_header = [x.replace(" ","_").replace("$","").replace("/","_").lower() for x in solar_header]
solar_header

In [ ]:
solar_json = []

for row in solar_data:
    
    row = dict(zip(solar_header, row))
    
    row['reporting_period'] = dates(row['reporting_period'])
    
    row['date_application_received'] = ymd(row['date_application_received'])
    
    try:
        row['date_completed'] = ymd(row['date_completed'])
    except:
        pass
    
    app_list0 = row['location_1'].split('\n')
    app_dict0 = dict(zip(['address', 'geo'], app_list0))
    
    row['location'] = app_dict0
    
    del row['location_1']
    
    for col in ['total_inverter_quantity', 'total_pv_module_quantity', 
                'project_cost', 'incentive','total_nameplate_kw_dc',
                'expected_kwh_annual_production']:
        row[col] = nums(row[col])
        
    row['current_date'] = datetime.datetime.now()
    
    solar_json.append(row)
        
        

In [ ]:
for k, v in solar_json[0].items():
    print(k, '>>>', v)

In [ ]:
pprint(solar_json[0], width=132, indent=2)

In [ ]:
with open('solar-data.json', 'w') as fp:
    json.dump(fp=fp, obj=solar_json, indent=2, default=json_util.default)

In [ ]:
with open('solar-data.json', 'r') as f:
    json_data = json.load(f, object_hook=json_util.object_hook)

In [ ]:
type(json_data)

In [ ]:
type(json_data[0])

In [ ]:
for k, v in json_data[0].items():
    print(k, '>>>', v)

In [ ]:
pprint(json_data[0], width=132, indent=2)